In [ ]:
# if the following command generates an error, you probably didn't enable
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

In [ ]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [ ]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

In [ ]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [ ]:
spark


In [ ]:
# TODO: ADD BUCKET NAME!
bucket_name = 'bucket_ir_100'

In [ ]:
# If some files are in the way you can rerun this.
full_path = f"gs://{bucket_name}/"
paths=[]
client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp") and not b.name.startswith("pr") and not b.name.startswith("word_counts"):
        paths.append(full_path+b.name)

In [ ]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("title", "id").rdd

In [ ]:
# Convert RDD to DataFrame
word_counts_df = doc_text_pairs.toDF(['id', 'title'])

# Write word counts to CSV in Google Cloud Storage
output_path = f'gs://{bucket_name}/id_title'
word_counts_df.repartition(1).write.csv(output_path, compression="gzip")